In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
! ls

AmesHousing.tsv  Basics.ipynb


In [6]:
ddf = pd.read_csv("AmesHousing.tsv", delimiter="\t")

In [9]:
df.head()

,Order PID MS SubClass MS Zoning Lot Frontage Lot Area Street Alley Lot Shape Land Contour Utilities Lot Config Land Slope Neighborhood Condition 1 Condition 2 Bldg Type House Style Overall Qual Overall Cond Year Built Year Remod/Add Roof Style Roof Matl Exterior 1st Exterior 2nd Mas Vnr Type Mas Vnr Area Exter Qual Exter Cond Foundation Bsmt Qual Bsmt Cond Bsmt Exposure BsmtFin Type 1 BsmtFin SF 1 BsmtFin Type 2 BsmtFin SF 2 Bsmt Unf SF Total Bsmt SF Heating Heating QC Central Air Electrical 1st Flr SF 2nd Flr SF Low Qual Fin SF Gr Liv Area Bsmt Full Bath Bsmt Half Bath Full Bath Half Bath Bedroom AbvGr Kitchen AbvGr Kitchen Qual TotRms AbvGrd Functional Fireplaces Fireplace Qu Garage Type Garage Yr Blt Garage Finish Garage Cars Garage Area Garage Qual Garage Cond Paved Drive Wood Deck SF Open Porch SF Enclosed Porch 3Ssn Porch Screen Porch Pool Area Pool QC Fence Misc Feature Misc Val Mo Sold Yr Sold Sale Type Sale Condition SalePrice
0,1\t0526301100\t020\tRL\t141\t31770\tPave\tNA\t...
1,2\t0526350040\t020\tRH\t80\t11622\tPave\tNA\tR...
2,3\t0526351010\t020\tRL\t81\t14267\tPave\tNA\tI...
3,4\t0526353030\t020\tRL\t93\t11160\tPave\tNA\tR...
4,5\t0527105010\t060\tRL\t74\t13830\tPave\tNA\tI...


In [12]:
df.columns

Index(['Order\tPID\tMS SubClass\tMS Zoning\tLot Frontage\tLot Area\tStreet\tAlley\tLot Shape\tLand Contour\tUtilities\tLot Config\tLand Slope\tNeighborhood\tCondition 1\tCondition 2\tBldg Type\tHouse Style\tOverall Qual\tOverall Cond\tYear Built\tYear Remod/Add\tRoof Style\tRoof Matl\tExterior 1st\tExterior 2nd\tMas Vnr Type\tMas Vnr Area\tExter Qual\tExter Cond\tFoundation\tBsmt Qual\tBsmt Cond\tBsmt Exposure\tBsmtFin Type 1\tBsmtFin SF 1\tBsmtFin Type 2\tBsmtFin SF 2\tBsmt Unf SF\tTotal Bsmt SF\tHeating\tHeating QC\tCentral Air\tElectrical\t1st Flr SF\t2nd Flr SF\tLow Qual Fin SF\tGr Liv Area\tBsmt Full Bath\tBsmt Half Bath\tFull Bath\tHalf Bath\tBedroom AbvGr\tKitchen AbvGr\tKitchen Qual\tTotRms AbvGrd\tFunctional\tFireplaces\tFireplace Qu\tGarage Type\tGarage Yr Blt\tGarage Finish\tGarage Cars\tGarage Area\tGarage Qual\tGarage Cond\tPaved Drive\tWood Deck SF\tOpen Porch SF\tEnclosed Porch\t3Ssn Porch\tScreen Porch\tPool Area\tPool QC\tFence\tMisc Feature\tMisc Val\tMo Sold\tYr Sold

In [8]:
def transform_features():
    return train

In [13]:
def select_features():
    return train['Gr_Liv_Area'], train['SalePrice']

In [14]:
def train_and_test():
    # select first 1460 rows from the data for train
    # rest for test
    pass